###**Тема проекта**
Аналитика шахматных партий, на основе предоставленных данных о игроках, дебюте, количества ходов и итоге


**Источник:** https://www.pgnmentor.com/files.html

> Открытая база данных партий в pgn нотации, в которой можно найти распределение партий по игрокам, дебютам, а так же соревнованиям. Сайт регулярно обновляется, последнее обновление партий май 2024.

#Сбор данных

Был выполнен парсинг сайта с шахматными нотациями, так как в данный момент сайт chess.com и другие аналогичные сервисы заблокированны для парсинга из России.

В ходе парсинга были трудности, в связи с необычностью сайта, и предоставлением самих ответов на реквесты, что обошлось с помощью скачивания зип файлов и дальнейшего их прогона через библиотеку chess (была найдена случайно), иначе было бы самоубийством пробовать находить данные через обычный парсер текстовых документов.

In [1]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00


In [2]:
from time import sleep
import re
import requests, zipfile, io # parsing
from bs4 import BeautifulSoup # parsing

import chess.pgn #чтобы нормально взять данные из партий
# documentation https://python-chess.readthedocs.io/en/latest/index.html
import csv
import os

from google.colab import drive

import pandas as pd

**Шаг 1**

Отправляем запрос на сайт и смотрим ответ на него.

> Ответом должен быть HTML код из которого мы и будем извлекать все нас интересующее


In [ ]:
urls = 'https://www.pgnmentor.com/files.html'
grab = requests.get(urls)
soup = BeautifulSoup(grab.text, 'html.parser')

> Так как сайт довольно специфичный, и сразу скачиваются файлы без предварительных ссылок, то находим все ссылки на сайте через тег "а" и тип данных "href".

In [ ]:
links = []

# файлик для ссылок
f = open("all_links.txt", "w")

# Проходимся по всем тагам 'a' и проверяем, что тип 'href'
for link in soup.find_all("a"):
    data = link.get('href')
    if data:
        links.append(data)
        f.write(data)
        f.write("\n")
f.close()

**Шаг 2**

Отбираем только опенинги, которые нам будут нужны (через регулярные выражения), и добавляем их в список. Так же запишем их в отдельный файл.
> Всего 232 различных опенинга (это информация по сайту, однаку если смотреть на код ECO обозначающий вариант дебюта их больше. Скорее всего будем брать как вариант сайта, потому что хот енкодить миллиард дебютов ~= смерть).

In [ ]:
myfiles=[]
for l in set(links):
    # то есть, любые знаки до "openings" и любые знаки после, с расширением zip
    if re.search(".*openings/.*\.zip$", str(l)):
        myfiles.append(l)

myfiles = sorted(myfiles)
f = open("openigs_links.txt", "w")
for i in myfiles:
    f.write(i)
    f.write("\n")
f.close()

**Шаг 3**

Скачиваем файлы по ссылке (по ссылке передается сразу файл в зип формате, поэтому используем zipfile). Были проблемы с расшифровкой, поэтому использовали io.BytesIO

In [ ]:
def download_file(url):
    path = "https://www.pgnmentor.com/"+url
    r = requests.get(path, allow_redirects=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("database/")


for url in myfiles:
    sleep(1) # чтобы не забанили за слишком частое обращение к серверу
    download_file(url)

Берем все названия скачанных файлов, хотя они уже были в myfiles но да ладно

In [ ]:
files = [f for f in os.listdir('database/')]

**Шаг 3**

Создаем табличку данных.

>Так как в некоторых данных есть запятая, то используем таб как разделитель.

*В ридере пгн файла (стандартные файлы для шахмат), функция read_game последовательно считывает каждую игру, если игр в файле больше нет, то выдает None. Поэтому используем while, для каждого файла.*

> Также нам критично наличие Elo игроков, для машинного обучения, поэтому без этих данных (например слишком старых, когда Elo еще не было) мы пропускаем игры, чем и облегчим данные (*спойлер: облегчить сильно не получилось, все равно их дофига*).

*Функция .ply() возвращает количество полуходов за партию (то есть нужно разделить на 2, чтобы получилось количество ходов, к которым мы привыкли).*



---


Оно парсит более 3 часов, помогите:

---



In [ ]:
with open("database2.csv", "w", newline="") as data:
    writer = csv.writer(data, delimiter='\t')

    header = ['Year', 'Month', 'Day', 'Event', 'Site', 'Round', 'White',
              'Black',
              'WhiteElo', 'BlackElo', "Opening", "ECO", "Moves number",
              "Result"]
    writer.writerow(header)

    for file in files:
        with open('database/'+file, encoding='unicode_escape') as pgn:
            game = chess.pgn.read_game(pgn)
            while game:
                if (game.headers["WhiteElo"] if ("WhiteElo" in game.headers) else "").isdigit() and \
               (game.headers["BlackElo"] if ("BlackElo" in game.headers) else "").isdigit():
                    new_row = []
                    new_row.append(int(game.headers["Date"][0:4]) if game.headers["Date"][0:4].isdigit() else game.headers["Date"][0:4])
                    new_row.append(int(game.headers["Date"][5:7]) if game.headers["Date"][5:7].isdigit() else game.headers["Date"][5:7])
                    new_row.append(int(game.headers["Date"][8:10]) if game.headers["Date"][8:10].isdigit() else game.headers["Date"][8:10])
                    new_row.append(game.headers["Event"])
                    new_row.append(game.headers["Site"])
                    new_row.append(game.headers["Round"])
                    new_row.append(game.headers["White"])
                    new_row.append(game.headers["Black"])
                    new_row.append(int(game.headers["WhiteElo"]))
                    new_row.append(int(game.headers["BlackElo"]))
                    new_row.append(file[:-4])
                    new_row.append(game.headers["ECO"] if ("ECO" in game.headers) else "-")
                    new_row.append(int(game.end().ply()/2))
                    new_row.append(game.headers["Result"])
                    writer.writerow(new_row)
                game = chess.pgn.read_game(pgn)

ERROR:chess.pgn:illegal san: 'Nxb5' in 2r1r1k1/p4pp1/2q4p/1p4b1/2n2N2/PR4PP/1PP3Q1/2BR3K w - - 4 30 while parsing <Game at 0x7a1b6baab5e0 ('Aleksandrov,A' vs. 'Rajesh,VA', '2014.05.26' at 'Bhubaneswar IND')>
ERROR:chess.pgn:illegal san: 'axb6' in r3k2r/3nbppp/1nppp3/p4PP1/1p1PP3/3PBNN1/PP5P/1K1R3R b k - 0 18 while parsing <Game at 0x7a1b6b712ce0 ('Dominguez,L' vs. 'Nilsson,N', '2002.07.16' at 'Copenhagen DEN')>
ERROR:chess.pgn:illegal san: 'Qd8' in 8/p2r4/2p3kp/R7/P7/1P1b1P2/4pKP1/4B3 w - - 7 54 while parsing <Game at 0x7a1b6b674e80 ('Slovak,Kilian' vs. 'Cifka,S', '2021.11.14' at 'Czech Republic CZE')>
ERROR:chess.pgn:illegal san: 'Qb3' in 3r1b1r/kbq1pp1p/pn4p1/1p1pP1B1/6P1/P4P2/1PP1BQ1P/2KR2NR w - - 4 18 while parsing <Game at 0x7a1b6b63e890 ('Yassibag,Ipek Asli' vs. 'Zengin,Hande', '2014.06.23' at 'Konya TUR')>
ERROR:chess.pgn:illegal san: 'Qxd2' in r3kn2/4bppr/p1bp4/qp2pPP1/4P3/P1N1B3/1PPQ4/2KR1BR1 w q - 0 19 while parsing <Game at 0x7a1b6bb03a60 ('Farago,S' vs. 'Li Hong', '2018.11.

**Информация о полученных данных (по столбцам):**

1.    Year - год игры партии
>*Возможно некоторые дебюты были сильнее в какое-то время, так как придумывались другие продолжения*
2.    Month - месяц
3.    Day - день игры
>*Визуально много партий без месяца и дня, особенно старые, поэтому скорее всего избавимся от этих столбцов*
4.    Event - турнир, на котором игралась партия
5.    Site - место игры
>*Некоторые в онлайн играют лучше или хуже, возможно связано с возможностью премува или невозможностью смотреть за реакцией соперника, также нуждается в проверке*
6.    Round - раунд турнира
7.    White - игрок за белых
8.    Black - игрок за белых
>*В основном за белых играю на победу, а за черных на ничью, обуславливается первым ходом белых, однако некоторые играют лучше за черных, но данную теорию надо будет проверить*
9.    WhiteElo - Эло (рейтинг) игрока за белых
10.   BlackElo - Эло (рейтинг) игрока за черных
>*Партии с путыми Эло уже выкинуты, иначе мало данных для анализа*
11.   Opening - название дебюта в соответствии с источником для исследования
12.   ECO - номер дебюта в общепринятой шахматной системе
>*Как и говорилось ранее, Opening и ECO незначительно отличаются друг от друга.*
13.   Moves Number - количество ходов в партии
14.   Result - итоговый результат партии
>1-0 - победа белых, 1/2-1/2 - ничья, 0-1 - победа черных

#Предварительная обработка

сами данные можно скачать по ссылке (более 100 мб): https://drive.google.com/file/d/113VKTy8jcDM3055uUOGRmWvTNV1eVbPo/view?usp=share_link

К сожалению из-за большой сложности в парсинге данных, мы не успели сделать препроцессинг даты, а так же ее первичный анализ.

>Однако несколько идей есть, как и распределение побед в партиях (относительно белых и черных), так же их распределение в онлайн и оффлайн, и относительно отдельных игроков.

Также интересно посмотреть насколько сильно влияет большая разница в рейтинге (более 100, или 150, так как партии по большей части сыграны гроссмейстерами, и разница в рейтинге у них не сильно решает).

>Интерес представляет и доминация некоторых отдельных дебютов в определенные года, поэтому нужно будет разбить года на отрезки по 5/10 лет, и посмотреть топ дебютов, и насколько они влияли на победу одних или других.

In [5]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/database2.csv', delimiter='\t')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Year,Month,Day,Event,Site,Round,White,Black,WhiteElo,BlackElo,Opening,ECO,Moves number,Result
0,1971.0,??,??,IBM,Amsterdam,6,"Scholl, Eduard C","Timman, Jan H",2430,2445,PetroffOther3,C49,43,0-1
1,1971.0,??,??,URS-ch39,Leningrad,?,"Geller, Efim P","Smyslov, Vassily",2615,2620,PetroffOther3,C43,11,1/2-1/2
2,1971.0,??,??,URS-ch39,Leningrad,?,"Shamkovich, Leonid","Smyslov, Vassily",2520,2620,PetroffOther3,C65,14,1/2-1/2
3,1971.0,??,??,CSR-ch,Luhacovice,4,"Jansa, Vlastimil","Kolarov, Atanas S",2450,2435,PetroffOther3,C43,31,1-0
4,1971.0,??,??,Malaga,Malaga,?,"Ciocaltea, Victor","Rogoff, Kenneth",2460,2410,PetroffOther3,C43,43,1-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900167,2023.0,12,30,World Blitz 2023,Samarkand UZB,16.101,"Bogdan,David","Amangeldy,Ernur",1671,2134,Sicilian2d6-4Qxd4,B53,99,0-1
2900168,2023.0,12,30,World Blitz 2023,Samarkand UZB,17.47,"Hovhannisyan,R","Gokerkan,Cem Kaan",2611,2523,Sicilian2d6-4Qxd4,B53,32,1-0
2900169,2023.0,12,30,World Blitz 2023,Samarkand UZB,17.101,"Bogdan,David","Kuandykuly,Danis",1671,2095,Sicilian2d6-4Qxd4,B53,47,0-1
2900170,2023.0,12,30,World Blitz 2023,Samarkand UZB,19.26,"Shimanov,A","Bocharov,D",2578,2518,Sicilian2d6-4Qxd4,B53,33,1-0
